In [6]:
import sys

# Add the willybears_crystal_balls directory to Python path
sys.path.append('/Users/williamfoote/Documents/GitHub/willybears_crystal_balls')

# Now you can import everything from the utils module
from utils import *


[2024-08-25 17:20:01,522 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: /Users/williamfoote/Documents/GitHub/willybears_crystal_balls/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
[2024-08-25 17:20:01,525 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=/Users/williamfoote/.matplotlib
[2024-08-25 17:20:01,526 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-08-25 17:20:01,526 DEBUG] [matplotlib.__init__.<module>] platform is darwin
[2024-08-25 17:20:01,558 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=/Users/williamfoote/.matplotlib
[2024-08-25 17:20:01,559 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from /Users/williamfoote/.matplotlib/fontlist-v330.json
[2024-08-25 17:20:01,559 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from /Users/williamfoote/.matplotlib/fontlist-v330.json


In [9]:
# Navigate relative to the current notebook location
sc = OAuth2(None, None, from_file='/Users/williamfoote/Documents/GitHub/willybears_crystal_balls/oauth2.json')


[2024-08-25 17:21:05,862 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-08-25 17:21:05,866 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 17966045.452457905
[2024-08-25 17:21:05,867 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2024-08-25 17:21:05,867 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [23]:
gm = yfa.Game(sc, 'nfl')
leagues = gm.league_ids()
lg = gm.to_league('449.l.135898')
teamkey = lg.team_key()
team = lg.to_team(teamkey)
# start_week = 14
week = 1

In [33]:
player_ids = pd.read_csv('/Users/williamfoote/Documents/GitHub/willybears_crystal_balls/player_ids.csv')
player_ids['sleeper_id'] = player_ids['sleeper_id'].astype('str')
player_ids['yahoo_id'] = player_ids['yahoo_id'].fillna(0).astype(int).astype('str')

In [34]:
free_agents = {}
positions = lg.positions()
positions_list = list(positions.keys())

for position in positions_list:
    if position not in ['BN', 'W/R/T', 'IR']:  # Exclude 'BN' (Bench) position
        # Get the list of free agents for the position
        agents = lg.free_agents(position)
        # Append each agent to the free_agents dictionary
        for agent in agents:
            free_agents[agent['player_id']] = agent

# Add players from waivers
waiver_players = lg.waivers()
for player in waiver_players:
    free_agents[player['player_id']] = player

# Convert to DataFrame
import pandas as pd
free_agents_df = pd.DataFrame.from_dict(free_agents, orient='index')
free_agents_df = free_agents_df.rename(columns={'player_id':'yahoo_id'})
free_agents_df['yahoo_id'] = free_agents_df['yahoo_id'].astype('str')

In [42]:
free_agents_df.loc[free_agents_df['name'] == 'Jalen Hurts',]

,yahoo_id,name,position_type,eligible_positions,percent_owned,status
32723,32723,Jalen Hurts,O,[QB],100,


In [46]:
# Step 1: Call the taken_players method to get the keepers
keepers_json = lg.taken_players()

# Step 2: Convert the JSON data to a pandas DataFrame
keepers_df = pd.DataFrame(keepers_json)

# Optional: Display the DataFrame or perform further operations
keepers_df

,player_id,name,position_type,eligible_positions,percent_owned,status
0,27535,Mike Evans,O,"[WR, W/R/T]",100,
1,28654,Raheem Mostert,O,"[RB, W/R/T]",94,
2,29369,Dak Prescott,O,[QB],97,
3,29399,Tyreek Hill,O,"[WR, W/R/T]",100,Q
4,30121,Christian McCaffrey,O,"[RB, W/R/T]",100,Q
5,30180,Alvin Kamara,O,"[RB, W/R/T]",97,
6,31017,Christian Kirk,O,"[WR, W/R/T]",92,Q
7,31868,Deebo Samuel Sr.,O,"[WR, W/R/T]",100,
8,32695,Brandon Aiyuk,O,"[WR, W/R/T]",99,
9,32704,Michael Pittman Jr.,O,"[WR, W/R/T]",97,


In [51]:
import pandas as pd

# Example function to determine the number of rounds
def get_total_rounds(positions_json):
    total_positions = sum(position['count'] for position in positions_json.values() if position['is_starting_position'] == 1)
    bench_positions = positions_json['BN']['count']
    total_rounds = total_positions + bench_positions
    return total_rounds

# Example function to generate the snake draft order
def generate_snake_draft_order(owners, total_rounds):
    draft_order = []
    for round_num in range(total_rounds):
        if round_num % 2 == 0:  # Even rounds (0-indexed), normal order
            round_order = owners
        else:  # Odd rounds, reverse order
            round_order = owners[::-1]
        for pick, owner in enumerate(round_order, start=1):
            draft_order.append({'round': round_num + 1, 'pick': pick, 'owner': owner})
    return pd.DataFrame(draft_order)

# Sample owners list and draft positions
owners = ['Owner1', 'Owner2', 'Owner3', 'Owner4']  # Example owners list

# Step 1: Fetch the positions and calculate total rounds
positions_json = lg.positions()
total_rounds = get_total_rounds(positions_json)

# Step 2: Generate snake draft order DataFrame
draft_df = generate_snake_draft_order(owners, total_rounds)

# Step 3: Adjust the draft DataFrame to account for keepers
keepers_json = lg.taken_players()  # Fetch the keepers data
keepers_df = pd.DataFrame(keepers_json)
keepers_df['keeper_round'] = keepers_df['keeper_round'].apply(lambda x: max(1, x - 1))  # Adjust round by subtracting 1

# Merge keepers into the draft DataFrame
for index, keeper in keepers_df.iterrows():
    keeper_owner = keeper['owner']  # Assuming we have owner information in keepers data
    keeper_round = keeper['keeper_round']
    
    # Find the row in draft_df that matches the keeper's round and owner
    mask = (draft_df['round'] == keeper_round) & (draft_df['owner'] == keeper_owner)
    draft_df.loc[mask, 'player'] = keeper['name']
    draft_df.loc[mask, 'is_keeper'] = True

# Ensure columns like player and is_keeper exist
if 'player' not in draft_df.columns:
    draft_df['player'] = None
if 'is_keeper' not in draft_df.columns:
    draft_df['is_keeper'] = False

# Step 4: Display the draft DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Draft DataFrame", dataframe=draft_df)


KeyError: 'keeper_round'

In [52]:
# Step 3: Adjust the draft DataFrame to account for keepers
keepers_json = lg.taken_players()  # Fetch the keepers data
keepers_df = pd.DataFrame(keepers_json)
keepers_df.

,player_id,name,position_type,eligible_positions,percent_owned,status
0,27535,Mike Evans,O,"[WR, W/R/T]",100,
1,28654,Raheem Mostert,O,"[RB, W/R/T]",94,
2,29369,Dak Prescott,O,[QB],97,
3,29399,Tyreek Hill,O,"[WR, W/R/T]",100,Q
4,30121,Christian McCaffrey,O,"[RB, W/R/T]",100,Q
5,30180,Alvin Kamara,O,"[RB, W/R/T]",97,
6,31017,Christian Kirk,O,"[WR, W/R/T]",92,Q
7,31868,Deebo Samuel Sr.,O,"[WR, W/R/T]",100,
8,32695,Brandon Aiyuk,O,"[WR, W/R/T]",99,
9,32704,Michael Pittman Jr.,O,"[WR, W/R/T]",97,


In [57]:
free_agents_df.loc[free_agents_df['name'] == 'CeeDee Lamb', ]

,yahoo_id,name,position_type,eligible_positions,percent_owned,status


In [62]:
lg.taken_players_cache

[{'player_id': 27535,
  'name': 'Mike Evans',
  'position_type': 'O',
  'eligible_positions': ['WR', 'W/R/T'],
  'percent_owned': 100,
  'status': ''},
 {'player_id': 28654,
  'name': 'Raheem Mostert',
  'position_type': 'O',
  'eligible_positions': ['RB', 'W/R/T'],
  'percent_owned': 94,
  'status': ''},
 {'player_id': 29369,
  'name': 'Dak Prescott',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 97,
  'status': ''},
 {'player_id': 29399,
  'name': 'Tyreek Hill',
  'status': 'Q',
  'position_type': 'O',
  'eligible_positions': ['WR', 'W/R/T'],
  'percent_owned': 100},
 {'player_id': 30121,
  'name': 'Christian McCaffrey',
  'status': 'Q',
  'position_type': 'O',
  'eligible_positions': ['RB', 'W/R/T'],
  'percent_owned': 100},
 {'player_id': 30180,
  'name': 'Alvin Kamara',
  'position_type': 'O',
  'eligible_positions': ['RB', 'W/R/T'],
  'percent_owned': 97,
  'status': ''},
 {'player_id': 31017,
  'name': 'Christian Kirk',
  'status': 'Q',
  'position_t